# Dynamic World: Uncertainty Quantification
This notebooks provides an example of using the conformalImageClassifier class by quantifying uncertainty for the Dynamic World dataset. This dataset contains 9. For each candidate class, a probability band is made readily available.

We use the validation data, not used in training the model, to calibrate a conformal classifier and then evaluate the model. Finally, we will use the calibrated classifier to run inference on a new (out-of-sample) image. This will be achieved in five steps.

**Step 1**: Load modules  
**Step 2**: Prepare Dynamic world data  
**Step 3**: Calibrate a conformal classifier  
**Step 4**: Evaluate a conformal classifier  
**Step 5**: Run inference on a new scene using the calibrated conformal classifier


## Step 1: Load Modules

In [1]:
%load_ext watermark
%load_ext autoreload

In [2]:
import ee

try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

from geeml.utils import eeprint

import sys
MODULE_FULL_PATH = r'C:\Users\coach\myfiles\postdoc\Uncertainty\code\GEEConformal\code'
sys.path.insert(1, MODULE_FULL_PATH)

from conformalClassifier import conformalImageClassifier

In [3]:
%watermark -v -m --iversions

Python implementation: CPython
Python version       : 3.9.13
IPython version      : 8.4.0

Compiler    : MSC v.1929 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 165 Stepping 2, GenuineIntel
CPU cores   : 12
Architecture: 64bit

sys: 3.9.13 | packaged by conda-forge | (main, May 27 2022, 16:50:36) [MSC v.1929 64 bit (AMD64)]
ee : 0.2



## Step 2: Prepare Dynamic World data

In [4]:
# Create a single polygon with a global extent
globalBounds = ee.Geometry.Polygon([-180, 90, 0, 90, 180, 90, 180, -90, 10, -90, -180, -90], None, False)

# List of probability band names
bands = ['water', 'trees', 'grass', 'flooded_vegetation',
'crops', 'shrub_and_scrub', 'built', 'bare', 'snow_and_ice']

# Load dynamic world validation tiles
dwl = ee.ImageCollection('projects/nina/GIS_synergy/Extent/DW_global_validation_tiles')
dwLabels = dwl.select([1], ['label']).map(lambda img: ee.Image(img.updateMask(# Select reference label band
    img.gt(0).And(img.lt(10))).subtract(1).copyProperties(img))# remove unmarked up areas and extra-class
    # hacky method to edit image property
    .set('joinindex', img.rename(img.getString('system:index')).regexpRename('^[^_]*_', '').bandNames().getString(0)))\
    .randomColumn(**{'seed': 42})# add random column

dwp = ee.ImageCollection("projects/ee-geethensingh/assets/UQ/DW_probs")
dwp = dwp.map(lambda img: ee.Image(img.rename(bands).selfMask()).copyProperties(img) #rename bands, mask 0 pixels
    # Hacky method to edit image property
    .set('joinindex', img.select([0]).rename(img.getString('id_no')).regexpRename('^[^_]*_', '').bandNames().getString(0)))
  

In [5]:
# Join label collection and probability collection on their 'joinindex' property. The propertyName parameter
# is the name of the property that references the joined image.
def indexJoin(collectionA, collectionB, propertyName):
    joined = ee.ImageCollection(ee.Join.saveFirst(propertyName).apply(**{
    'primary': collectionA,
    'secondary': collectionB,
    'condition': ee.Filter.equals(**{
      'leftField': 'joinindex',
      'rightField': 'joinindex'})
      })) 
    # Merge the bands of the joined image.
    return joined.map(lambda image: image.addBands(ee.Image(image.get(propertyName))))

dwCombined = indexJoin(dwLabels, dwp, 'probImage')

## Step 3: Calibrate conformal classifier

We use 80% of the image chips for calibrating the conformal classifier and the remaining 20% for evaluating the conformal classifier. Note, the evaluation phase is more compute heavy than the calibration stage. All analyses are performed at the native spatial resolution of dynamic world (10 m). We specify an alpha value of 10% i.e., we can tolerate a 10% error level or in other words we are satisfied if 10% of pixels do not contain the actual label within their prediction sets.

In [10]:
%autoreload 2

In [15]:
SCALE = 10 #Used to compute Eval metrics
ALPHA = 0.1 #1-ALPHA corresponds to required coverage. For example, 0.1 for 90% coverage
SPLIT = 0.95 #Split used for calibration and test data (for example, 0.8 corresponds to 80% calibration data)
LABEL = 'label' #Band name for reference label band

# Intialise conformal Image classifier
cc = conformalImageClassifier(data = dwCombined, scale = SCALE,
                            bands = bands, alpha = ALPHA, split = SPLIT,
                            label = LABEL, version = 'demoDW_06012024')
# Calibrate conformal Image classifier
eeprint(cc.calibrate())

## Step 4: Evaluate conformal classifier
During evaluation on the test set, we use two metrics
1. The average set size (The sum of the lengths for all prediction sets, divided by the number of test pixels)
2. The empirical marginal coverage (The proportion of sets that contain the reference labels)

The ideal average set size is 1. A value less than 1 suggests many empty sets while, in this case, a value closer to 9 indicates a statistically inefficient conformal predictor. We have a value of 2.29 which is a rather statistically efficient conformal predictor.

The empirical marginal coverage should be close to the the required coverage (0.9 in this case). We have a value of 0.94, which is greater than our required coverage. This suggests that the conformal predictor could be slightly more efficient( have a slightly smaller average set size and still satisfy our required coverage)

In [16]:
eeprint(cc.evaluate())

Average set size: 2.29
Empirical (marginal) coverage: 0.94


## Step 5: Inference 
We quantify uncertainty for an unseen image using a calibrated conformal classifier.

In [21]:
import geemap
Map = geemap.Map()
geometry = ee.Geometry.Polygon(
        [[[-124.58837360718884, 42.24132567361335],
          [-124.58837360718884, 32.1623568470788],
          [-113.95360798218884, 32.1623568470788],
          [-113.95360798218884, 42.24132567361335]]], None, False)
Map.addLayer(geometry)
Map.centerObject(geometry, 5)
Map

Map(center=[37.12915465149783, -119.27099079468887], controls=(WidgetControl(options=['position', 'transparent…

<IPython.core.display.Javascript object>

vis_params = {'bands': ['setLength'], 'palette': ['#440154', ' #440256', ' #450457', ' #450559', ' #46075a', ' #46085c', ' #460a5d', ' #460b5e', ' #470d60', ' #470e61', ' #471063', ' #471164', ' #471365', ' #481467', ' #481668', ' #481769', ' #48186a', ' #481a6c', ' #481b6d', ' #481c6e', ' #481d6f', ' #481f70', ' #482071', ' #482173', ' #482374', ' #482475', ' #482576', ' #482677', ' #482878', ' #482979', ' #472a7a', ' #472c7a', ' #472d7b', ' #472e7c', ' #472f7d', ' #46307e', ' #46327e', ' #46337f', ' #463480', ' #453581', ' #453781', ' #453882', ' #443983', ' #443a83', ' #443b84', ' #433d84', ' #433e85', ' #423f85', ' #424086', ' #424186', ' #414287', ' #414487', ' #404588', ' #404688', ' #3f4788', ' #3f4889', ' #3e4989', ' #3e4a89', ' #3e4c8a', ' #3d4d8a', ' #3d4e8a', ' #3c4f8a', ' #3c508b', ' #3b518b', ' #3b528b', ' #3a538b', ' #3a548c', ' #39558c', ' #39568c', ' #38588c', ' #38598c', ' #375a8c', ' #375b8d', ' #365c8d', ' #365d8d', ' #355e8d', ' #355f8d', ' #34608d', ' #34618d', ' #

In [25]:
# Create a compsoite dynamic world image over our geometry area. We use a first non-null
#  composite which selects the first valid pixel in our specified time range.

#  Data preparation - Spatio-temporal filtering
dwFiltered = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")\
.filterDate('2020-01-01', '2021-01-01')\
.filterBounds(geometry)\
.reduce(ee.Reducer.firstNonNull())\
.rename(ee.List(bands).add('label')).aside(eeprint)

In [27]:
# Perform inference
uqImage = cc.predict(dwFiltered)
# Specify visualisation parameters
vis_params = {'bands': ['setLength'], 'palette': ['#440154', ' #440256', ' #450457', ' #450559', ' #46075a', ' #46085c', ' #460a5d', ' #460b5e', ' #470d60', ' #470e61', ' #471063', ' #471164', ' #471365', ' #481467', ' #481668', ' #481769', ' #48186a', ' #481a6c', ' #481b6d', ' #481c6e', ' #481d6f', ' #481f70', ' #482071', ' #482173', ' #482374', ' #482475', ' #482576', ' #482677', ' #482878', ' #482979', ' #472a7a', ' #472c7a', ' #472d7b', ' #472e7c', ' #472f7d', ' #46307e', ' #46327e', ' #46337f', ' #463480', ' #453581', ' #453781', ' #453882', ' #443983', ' #443a83', ' #443b84', ' #433d84', ' #433e85', ' #423f85', ' #424086', ' #424186', ' #414287', ' #414487', ' #404588', ' #404688', ' #3f4788', ' #3f4889', ' #3e4989', ' #3e4a89', ' #3e4c8a', ' #3d4d8a', ' #3d4e8a', ' #3c4f8a', ' #3c508b', ' #3b518b', ' #3b528b', ' #3a538b', ' #3a548c', ' #39558c', ' #39568c', ' #38588c', ' #38598c', ' #375a8c', ' #375b8d', ' #365c8d', ' #365d8d', ' #355e8d', ' #355f8d', ' #34608d', ' #34618d', ' #33628d', ' #33638d', ' #32648e', ' #32658e', ' #31668e', ' #31678e', ' #31688e', ' #30698e', ' #306a8e', ' #2f6b8e', ' #2f6c8e', ' #2e6d8e', ' #2e6e8e', ' #2e6f8e', ' #2d708e', ' #2d718e', ' #2c718e', ' #2c728e', ' #2c738e', ' #2b748e', ' #2b758e', ' #2a768e', ' #2a778e', ' #2a788e', ' #29798e', ' #297a8e', ' #297b8e', ' #287c8e', ' #287d8e', ' #277e8e', ' #277f8e', ' #27808e', ' #26818e', ' #26828e', ' #26828e', ' #25838e', ' #25848e', ' #25858e', ' #24868e', ' #24878e', ' #23888e', ' #23898e', ' #238a8d', ' #228b8d', ' #228c8d', ' #228d8d', ' #218e8d', ' #218f8d', ' #21908d', ' #21918c', ' #20928c', ' #20928c', ' #20938c', ' #1f948c', ' #1f958b', ' #1f968b', ' #1f978b', ' #1f988b', ' #1f998a', ' #1f9a8a', ' #1e9b8a', ' #1e9c89', ' #1e9d89', ' #1f9e89', ' #1f9f88', ' #1fa088', ' #1fa188', ' #1fa187', ' #1fa287', ' #20a386', ' #20a486', ' #21a585', ' #21a685', ' #22a785', ' #22a884', ' #23a983', ' #24aa83', ' #25ab82', ' #25ac82', ' #26ad81', ' #27ad81', ' #28ae80', ' #29af7f', ' #2ab07f', ' #2cb17e', ' #2db27d', ' #2eb37c', ' #2fb47c', ' #31b57b', ' #32b67a', ' #34b679', ' #35b779', ' #37b878', ' #38b977', ' #3aba76', ' #3bbb75', ' #3dbc74', ' #3fbc73', ' #40bd72', ' #42be71', ' #44bf70', ' #46c06f', ' #48c16e', ' #4ac16d', ' #4cc26c', ' #4ec36b', ' #50c46a', ' #52c569', ' #54c568', ' #56c667', ' #58c765', ' #5ac864', ' #5cc863', ' #5ec962', ' #60ca60', ' #63cb5f', ' #65cb5e', ' #67cc5c', ' #69cd5b', ' #6ccd5a', ' #6ece58', ' #70cf57', ' #73d056', ' #75d054', ' #77d153', ' #7ad151', ' #7cd250', ' #7fd34e', ' #81d34d', ' #84d44b', ' #86d549', ' #89d548', ' #8bd646', ' #8ed645', ' #90d743', ' #93d741', ' #95d840', ' #98d83e', ' #9bd93c', ' #9dd93b', ' #a0da39', ' #a2da37', ' #a5db36', ' #a8db34', ' #aadc32', ' #addc30', ' #b0dd2f', ' #b2dd2d', ' #b5de2b', ' #b8de29', ' #bade28', ' #bddf26', ' #c0df25', ' #c2df23', ' #c5e021', ' #c8e020', ' #cae11f', ' #cde11d', ' #d0e11c', ' #d2e21b', ' #d5e21a', ' #d8e219', ' #dae319', ' #dde318', ' #dfe318', ' #e2e418', ' #e5e419', ' #e7e419', ' #eae51a', ' #ece51b', ' #efe51c', ' #f1e51d', ' #f4e61e', ' #f6e620', ' #f8e621', ' #fbe723', ' #fde725'], 'min': 0.0, 'max': 9.0}
# Visualise set length image i.e., the length of each pixels set. a higher value corresponds to a more uncertain prediction.
Map.addLayer(uqImage, vis_params, 'Uncertainty (set Length)')